In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import cudf
from cuml.preprocessing import LabelEncoder
import pandas as pd
import joblib

In [1]:
def create_inference_ui(df, model, scaler, label_encoders):

    ev_types = df['Electric Vehicle Type'].unique().to_pandas().tolist()
    utilities = df['Electric Utility'].unique().to_pandas().tolist()
    
    ev_types.sort()
    utilities.sort()
    
    style = {'description_width': '150px'}
    layout = widgets.Layout(width='500px')
    
    ev_type_dropdown = widgets.Dropdown(
        options=ev_types,
        description='EV Type:',
        style=style,
        layout=layout
    )
    
    electric_range = widgets.FloatText(
        value=200.0,
        description='Electric Range (miles):',
        min=0,
        max=500,
        style=style,
        layout=layout
    )
    
    utility_dropdown = widgets.Dropdown(
        options=utilities,
        description='Electric Utility:',
        style=style,
        layout=layout
    )
    
    output = widgets.Output()
    
    predict_button = widgets.Button(
        description='Predict CAFV Eligibility',
        button_style='info',
        layout=widgets.Layout(width='500px')
    )
    
    style_html = """
    <style>
        .prediction-box {
            padding: 20px;
            margin: 20px 0;
            border-radius: 10px;
            font-family: Arial, sans-serif;
        }
        .eligible {
            background-color: #d4edda;
            border: 1px solid #c3e6cb;
            color: #155724;
        }
        .not-eligible {
            background-color: #f8d7da;
            border: 1px solid #f5c6cb;
            color: #721c24;
        }
        .prediction-title {
            font-size: 18px;
            font-weight: bold;
            margin-bottom: 10px;
        }
        .prediction-details {
            margin-top: 15px;
            font-size: 14px;
        }
    </style>
    """
    
    def make_prediction(b):
        with output:
            output.clear_output()
            input_data = {
                'Electric Vehicle Type': [ev_type_dropdown.value],
                'Electric Range': [float(electric_range.value)],
                'Electric Utility': [utility_dropdown.value]
            }
            
            input_df = cudf.DataFrame(input_data)
            
            encoded_data = {
                'Electric Range': input_df['Electric Range'].astype('float32')
            }
            
            for col in ['Electric Vehicle Type', 'Electric Utility']:
                le = label_encoders[col]
                encoded_val = le.transform(input_df[col].astype('str'))
                encoded_data[f'{col}_encoded'] = encoded_val.astype('float32')
            
            X = cudf.DataFrame(encoded_data)
            
            X_scaled = scaler.transform(X)
            
            prediction = model.predict(X_scaled)
            probability = float(prediction[0])
            is_cafv = probability > 0.5
            
            display(HTML(style_html))
            
            result_class = 'eligible' if is_cafv else 'not-eligible'
            result_html = f"""
            <div class="prediction-box {result_class}">
                <div class="prediction-title">
                    {'CAFV Eligible' if is_cafv else 'Not CAFV Eligible'}
                </div>
                <div class="prediction-details">
                    <strong>Confidence:</strong> {probability:.2%}<br>
                    <strong>Input Details:</strong><br>
                    • Vehicle Type: {ev_type_dropdown.value}<br>
                    • Electric Range: {electric_range.value} miles<br>
                    • Electric Utility: {utility_dropdown.value}
                </div>
            </div>
            """
            display(HTML(result_html))
    
    predict_button.on_click(make_prediction)
    
    title = HTML("""
    <h2 style='
        color: #2c3e50;
        font-family: Arial, sans-serif;
        padding: 10px 0;
        margin-bottom: 20px;
        border-bottom: 2px solid #3498db;
    '>CAFV Eligibility Predictor</h2>
    """)
    
    display(title)
    display(widgets.VBox([
        ev_type_dropdown,
        electric_range,
        utility_dropdown,
        widgets.Box([predict_button], layout=widgets.Layout(margin='20px 0')),
        output
    ], layout=widgets.Layout(
        margin='10px 0',
        padding='20px',
        border='1px solid #eee',
        border_radius='10px'
    )))

In [ ]:
create_inference_ui(df, model, scaler, label_encoders)